# Clustering Crypto

In [3]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [ ]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [ ]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [68]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [63]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply\

In [69]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df["IsTrading"] == True]

In [53]:
# Keep only cryptocurrencies with a working algorithm

In [70]:
# Remove the "IsTrading" column
crypto_df.drop("IsTrading", axis=1, inplace=True)

In [71]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)


In [72]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"] > 0]

In [73]:
# Drop rows where there are 'N/A' text values
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [74]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_df = pd.DataFrame(data=crypto_df, columns=["CoinName"])

In [76]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)

In [77]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [79]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(crypto_scaled)

In [81]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.346904,0.967780,-0.562070
404,-0.330291,0.967647,-0.562178
1337,2.305010,1.578966,-0.587608
BTC,-0.136639,-1.239336,0.219427
ETH,-0.148881,-1.995774,0.417693
LTC,-0.174999,-1.163097,0.004900
DASH,-0.382497,1.236777,-0.529772
XMR,-0.128491,-2.185436,0.381098
ETC,-0.147327,-1.995904,0.417701
ZEC,-0.161450,-2.039052,0.405215


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [87]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model


# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
